## Sistema Inteligente de Perguntas e Respostas para Documentos PDF

Este notebook apresenta o desenvolvimento de um **sistema inteligente de Perguntas e Respostas aplicado a arquivos com formato PDF**.

A solução foi inspirada e baseada no artigo “How Extract Data from PDF Using LangChain and Mistral” de Jose Chipana, que demonstra o uso de LangChain, embeddings e modelos de linguagem para extrair e consultar informações de PDFs.

Foram utilizados os pdfs da disciplina como dados de entrada para o sistema

Referencias:

[How extract data from PDF using LangChain and Mistral](https://medium.com/@chipanajose/how-extract-data-from-pdf-using-langchain-and-mistral-74b252fd88a0)

[all-mpnet-base-v2 - modelo de embeddings](https://huggingface.co/sentence-transformers/all-mpnet-base-v2)

[guide to evaluate RAG](https://www.evidentlyai.com/llm-guide/rag-evaluation)


[ library for evaluating and testing Large Language Model (LLM) applications](https://docs.ragas.io/en/stable/getstarted/#step-4-run-your-evaluation)




In [ ]:
!pip install -U langchain-core langchain-community langchain-mistralai langchain-text-splitters faiss-cpu ragas pypdf --quiet


In [ ]:
from langchain_mistralai import ChatMistralAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from google.colab import files
from langchain_core.runnables import RunnableMap
from operator import itemgetter


In [ ]:
#fazendo upload
uploaded = files.upload()
#pega nomes
pdf_paths = list(uploaded.keys())



In [ ]:
#lendo os pdfs e transformando em list[document], cada document tem page_content e metada
documents = []

for path in pdf_paths:
    loader = PyPDFLoader(path)
    docs = loader.load()
    documents.extend(docs)

len(documents)

**Divisão do Texto em Chunks**

Antes de alimentar o sistema RAG, o texto original é dividido em partes menores para facilitar a indexação e melhorar a recuperação de informações.

Utilizamos o RecursiveCharacterTextSplitter com os seguintes parâmetros:

- **chunk_size = 1000** caracteres  
- **chunk_overlap = 150** caracteres de sobreposição

Essas configurações ajudam a manter a continuidade sem perder contexto entre os trechos.

In [ ]:
#dividindo o texto em chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150
)


chunks = text_splitter.split_documents(documents)
len(chunks)

**Criação dos Embeddings**

 Cada chunk de texto é transformado em uma representação vetorial, permitindo que o sistema realize buscas semânticas eficientes.


Foi utilizado **sentence-transformers/all-mpnet-base-v2**


Esse modelo é amplamente usado em tarefas de similaridade semântica por apresentar alto desempenho em benchmarks como o STS (Semantic Textual Similarity).

In [ ]:
#criando os embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2"
)

**Armazenamento dos Embeddings com FAISS**

Para permitir buscas rápidas e eficientes pelos vetores gerados, utilizamos o **FAISS**, uma biblioteca otimizada para operações de similaridade em grandes coleções de embeddings.



In [ ]:
#db eficiente para armazenar embeddings
vector_store = FAISS.from_documents(chunks, embeddings)

retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


**Instanciando  Mistral**

O modelo utilizado no pipeline RAG é o **Mistral Small**, acessado via API.

A instanciação do LLM é feita através do nome do modelo e a da chave de acesso.

 - A chave foi deixada explícita apenas para fins de teste, já que expira em um dia.  


In [ ]:
#instancia modelo, deixei a chave porque ela vence em um dia
llm = ChatMistralAI( model="mistral-small-latest", api_key="key" )

In [ ]:
#testando mistral
resp = llm.invoke("O que é uma rede neural?")
print(resp.content)


In [ ]:
#criando prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Você é um assistente especializado extração e busca de respostas em pdfs/ ."),
    ("human", "Use os trechos abaixo para responder a pergunta:\n\n{context}\n\nPergunta: {question}")
])

In [ ]:
#formantando doc
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

O pipeline abaixo utiliza **RunnableMap** para estruturar o fluxo do RAG.
Ele realiza três etapas principais:

1. **Recuperação de contexto** a partir da pergunta.
2. **Geração da resposta** usando o LLM com o contexto recuperado.
3. **Formatação da saída final** contendo pergunta, resposta e contexto.

In [ ]:


#pipeline que implementa RAG
rag_chain = (
    RunnableMap({
        "context": itemgetter("question") | retriever | format_docs,
        "question": itemgetter("question"),
    })
    | {
        "answer_output": prompt | llm,
        "question": itemgetter("question"),
        "context": itemgetter("context"),
    }
    | RunnableMap({
        "question": itemgetter("question"),
        "answer": lambda x: x["answer_output"].content,
        "context": itemgetter("context"),
    })
)

**Execução do Pipeline RAG**

Nesta etapa, avaliamos o comportamento do pipeline RAG utilizando o método invoke.A partir de cada questão enviada, o pipeline retorna:

- **Pergunta enviada ao modelo**
- **Resposta gerada pelo LLM**
- **Contextos recuperados pelo retriever**




In [ ]:

resposta = rag_chain.invoke({"question": "Do que falam os slides?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])



In [ ]:
resposta = rag_chain.invoke({"question": "O que é convolução sobre volumes ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:
resposta = rag_chain.invoke({"question": "Qual slide fala sobre DeepFace ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])




In [ ]:
resposta = rag_chain.invoke({"question": "O que fala sobre Extração/Geração de Respostas de um Texto?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])




In [ ]:
resposta = rag_chain.invoke({"question": "Qual é a efiencia do Transformer ?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])




In [ ]:
resposta = rag_chain.invoke({"question": "Como funciona enconder e decoder?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:
resposta = rag_chain.invoke({"question": "Qual a arquitetura das RNNS?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])



In [ ]:
resposta = rag_chain.invoke({"question": "Em qual pagina fala sobre a função de ativação Softmax?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])



In [ ]:
resposta = rag_chain.invoke({"question": "O que é Rede Neural Convolucional?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:
resposta = rag_chain.invoke({"question": "Como é arquitetura do GPT?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])



#### Avaliação do RAG

In [ ]:
from ragas.metrics import (answer_correctness,faithfulness, context_recall,answer_relevancy)
from ragas import evaluate
from datasets import Dataset



In [ ]:
#pega questions, respostas do rag, groud_truth, contex_retrieved

questions = []
rag_answers = []
ground_truths = []
retrieved_contexts_list = []

for gt_entry in ground_truth_dataset:
    question = gt_entry["question"]
    ground_truth = gt_entry["answer"]

    resposta = rag_chain.invoke({"question": question})
    questions.append(question)
    rag_answers.append(resposta["answer"])
    ground_truths.append(ground_truth)
    contexts_list = resposta["context"].split('\n\n')
    retrieved_contexts_list.append(contexts_list)

In [ ]:



eval_data = {
    "question": questions,
    "answer": rag_answers,
    "ground_truth": ground_truths,
    "contexts": retrieved_contexts_list
}

dataset = Dataset.from_dict(eval_data)



In [ ]:
result = evaluate(
    dataset,
    metrics=[
        answer_correctness,   # equivalente a acurácia
        faithfulness,         # equivalente a precisão
        context_recall,       # equivalente a recall
        answer_relevancy      # equivalente a F1
    ],
    llm=llm,
    embeddings=embeddings
)

print(result)

### Análise das Métricas

As metricas obtidas na avaliação do sistema RAG utilizando o modelo **Mistral** indicam um desempenho moderado.

**Answer Correctness:** 0.6494

O modelo gera respostas semanticamente coerentes com o ground truth em 68% dos casos. Esse valor é esperado para um modelo de porte médio e um RAG simples.

**Faithfulness:**  0.7250

As respostas estão razoavelmente alinhadas aos trechos recuperados.O  modelo não alucina em excesso, mas ainda pode melhorar.

**Context Recall:**  0.50
Apenas metade das informações relevantes foram recuperadas pelo retriever. Mostra que o retriever é um ponto no RAG que precisa de melhoria.


**Answer Relevancy:** 0.5995
As respostas são moderamente relevantes para a pergunta.



### Comparação experimentos

###



In [ ]:
#troca embedding para experimento
embeddings_base = HuggingFaceEmbeddings(
    model_name="bert-base-uncased"
)

In [ ]:
#db eficiente para armazenar embeddings
vector_store_experiment = FAISS.from_documents(chunks, embeddings_base)

retriever_experiment = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)


In [ ]:
#pipeline que implementa RAG
rag_chain = (
    RunnableMap({
        "context": itemgetter("question") | retriever_experiment | format_docs,
        "question": itemgetter("question"),
    })
    | {
        "answer_output": prompt | llm,
        "question": itemgetter("question"),
        "context": itemgetter("context"),
    }
    | RunnableMap({
        "question": itemgetter("question"),
        "answer": lambda x: x["answer_output"].content,
        "context": itemgetter("context"),
    })
)

In [ ]:

resposta = rag_chain.invoke({"question": "Do que falam os slides?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:
resposta = rag_chain.invoke({"question": "O que é convolução sobre volumes ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:
resposta = rag_chain.invoke({"question": "Qual slide fala sobre DeepFace ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])




In [ ]:
resposta = rag_chain.invoke({"question": "Qual slide fala sobre DeepFace ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])

In [ ]:
resposta = rag_chain.invoke({"question": "O que fala sobre Extração/Geração de Respostas de um Texto?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:

resposta = rag_chain.invoke({"question": "Qual é a efiencia do Transformer ?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])

In [ ]:
resposta = rag_chain.invoke({"question": "Como funciona enconder e decoder?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:

resposta = rag_chain.invoke({"question": "Qual a arquitetura das RNNS?"})

print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])

In [ ]:
resposta = rag_chain.invoke({"question": "Em qual pagina fala sobre a função de ativação Softmax?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:

resposta = rag_chain.invoke({"question": "O que é Rede Neural Convolucional?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])


In [ ]:
resposta = rag_chain.invoke({"question": "Como é arquitetura do GPT?"})


print("Pergunta:", resposta["question"])
print("Resposta:", resposta["answer"])
print("Contextos:", resposta["context"])

### Avaliando outra config

In [ ]:
#pega questions, respostas do rag, groud_truth, contex_retrieved

questions = []
rag_answers = []
ground_truths = []
retrieved_contexts_list = []

for gt_entry in ground_truth_dataset:
    question = gt_entry["question"]
    ground_truth = gt_entry["answer"]

    resposta = rag_chain.invoke({"question": question})
    questions.append(question)
    rag_answers.append(resposta["answer"])
    ground_truths.append(ground_truth)
    contexts_list = resposta["context"].split('\n\n')
    retrieved_contexts_list.append(contexts_list)

In [ ]:
eval_data_experiment = {
    "question": questions,
    "answer": rag_answers,
    "ground_truth": ground_truths,
    "contexts": retrieved_contexts_list
}

dataset = Dataset.from_dict(eval_data_experiment)



In [ ]:

result = evaluate(
    dataset,
    metrics=[
        answer_correctness,   # equivalente a acurácia
        faithfulness,         # equivalente a precisão
        context_recall,       # equivalente a recall
        answer_relevancy      # equivalente a F1
    ],
    llm=llm,
    embeddings=embeddings
)

print(result)


**Análise das Métricas**



Foi feita uma execução do experimento RAG, com mudança de embeddings para o modelo de embeddings
[bert-base-uncased](https://huggingface.co/google-bert/bert-base-uncased).




As métricas obtidas na nova avaliação do sistema RAG utilizando o modelo Mistral mostram uma performance moderada, com algumas melhorias e outras quedas em relação à execução anterior.



**Answer Correctness:** 0.6063

O modelo produziu respostas semanticamente compatíveis com o ground truth em cerca de 61% dos casos.
O desempenho caiu um pouco em relação ao experimento anterior, devido à mudança nos embeddings ou ao conjunto de chunks recuperados. Mesmo assim o resultado continua dentro do esperado para um pipeline RAG simples.

**Faithfulness:** 0.8400

O aumento de  Faithfulness sugere que o embedding atual ajudou o modelo a usar melhor o contexto que recebe. Foi a melhor métrica desse experimento.

**Context Recall:** 0.50

Assim como antes, apenas 50% das informações realmente relevantes foram recuperadas.


**Answer Relevancy**: 0.5667

O valor aumentou em relação ao experimento anterior (0.36 para 0.56), indicando que: as respostas agora estão mais conectadas às perguntas, o modelo está interpretando melhor o que o usuário deseja, e o novo embedding contribuiu para fornecer contexto mais alinhado as consultas.



### Comparação entre os dois modelos de embeddings e justificativa dos resultados


A comparação entre os dois experimentos mostra que a escolha do embedding impacta diretamente o desempenho do RAG.
O **all-mpnet-base-v2** apresentou maior answer correctness, indicando respostas mais próximas ao ground truth, mas teve menor faithfulness e answer relevancy, sugerindo que o contexto recuperado nem sempre era adequado.

No **bert-base-uncased**, houve melhora clara em faithfulness e answer relevancy, mostrando que o modelo passou a usar melhor o contexto e a alucinar menos, embora a correção total tenha caido um pouco.

Em ambos os casos, o context recall permaneceu em 0.50, indicando que o principal limite do sistema está na recuperação de documentos, não no LLM ou no embedding.